In [67]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import tree
import random
import math

In [68]:
hoteltest = pd.read_csv('hoteltest_testing_set.csv')
hoteltrain = pd.read_csv('hoteltest_training_set.csv')

In [69]:
hoteltest.dtypes
hoteltrain.dtypes

srch_id                          int64
site_id                          int64
visitor_location_country_id      int64
prop_country_id                  int64
prop_id                          int64
prop_starrating                  int64
prop_review_score              float64
prop_brand_bool                  int64
prop_location_score1           float64
prop_location_score2           float64
position                         int64
promotion_flag                   int64
srch_destination_id              int64
srch_length_of_stay              int64
srch_booking_window              int64
srch_adults_count                int64
srch_children_count              int64
srch_room_count                  int64
srch_saturday_night_bool         int64
srch_query_affinity_score       object
orig_destination_distance        int64
random_bool                      int64
click_bool                       int64
booking_bool                     int64
date_time_split                 object
srch_month               

In [70]:
hoteltest.drop(hoteltest.select_dtypes(['object']), inplace=True, axis=1)
hoteltrain.drop(hoteltrain.select_dtypes(['object']), inplace=True, axis=1)

In [71]:
hoteltest_prop_id = hoteltest[['prop_id']]
hoteltest_srch_id = hoteltest[['srch_id']]
hoteltrain_prop_id = hoteltrain[['prop_id']]
hoteltrain_srch_id = hoteltrain[['srch_id']]
hoteltest = hoteltest.drop(columns=['prop_id', 'srch_id'])
hoteltrain = hoteltrain.drop(columns=['prop_id', 'srch_id'])

In [72]:
properties = list(set().union(list(hoteltest.columns),list(hoteltrain.columns)))

In [75]:
def get_letor_row(df, prop_id, srch_id, entry):
    target = prop_id.iloc[entry][0]
    qid = "qid:" + str(srch_id.iloc[entry][0])
    features = []
    for i, feature in enumerate(properties):
        try: 
            value = df.loc[entry, feature]
        except:
            pass
        else:
            features.append(str(i + 1) + ":" + str(value))
    features = " ".join(features)
    return " ".join([str(target),qid,features])

In [ ]:
file = open("../LambdaMart/test.txt", "w")
for entry in range(len(hoteltest)):
    print(get_letor_row(hoteltest, hoteltest_prop_id, hoteltest_srch_id, entry), file=file)
file.close()

In [ ]:
file = open("../LambdaMart/train.txt", "w")
for entry in range(len(hoteltrain)):
    print(get_letor_row(hoteltrain, hoteltrain_prop_id, hoteltrain_srch_id, entry), file=file)
file.close()

In [66]:
hoteltest.dtypes

site_id                        int64
visitor_location_country_id    int64
prop_country_id                int64
prop_starrating                int64
prop_brand_bool                int64
promotion_flag                 int64
srch_destination_id            int64
srch_length_of_stay            int64
srch_booking_window            int64
srch_adults_count              int64
srch_children_count            int64
srch_room_count                int64
srch_saturday_night_bool       int64
orig_destination_distance      int64
random_bool                    int64
srch_month                     int64
srch_year                      int64
booking_month                  int64
vis_star                       int64
vis_usd                        int64
prop_hist_price                int64
cat_price_usd                  int64
comps_perc_diff                int64
dtype: object